# JEM092 Asset Pricing - Homework 1

<span style='background:yellow'> **Group : 20107894**</span>

Team Member:

Yanqi Guan : 20107894@fsv.cuni.cz

Lin Zhang : 15845542@fsv.cuni.cz

Jiayi Zeng : 46566215@fsv.cuni.cz


### Task 1 - Data download (8 pts)
In the previous step, you were assigned firms for which you will get the data. You will download 
* daily adjusted close price and volume data from www.finance.yahoo.com
* Market Capitalization and Book Value per Share data from www.macrotrends.net
* Sample period: 01.01.2007 - 28.02.2022 

In [ ]:
# Setup environment
Sys.setenv(LANG = "en")
Sys.setlocale("LC_TIME", "English")
#options(warn = -1)  # suppressing warnings

if (!require(quantmod)) install.packages('quantmod')
if (!require(httr)) install.packages('httr')
if (!require(rvest)) install.packages('rvest')
if (!require(jsonlite)) install.packages('jsonlite')


library(quantmod)
library(httr)
library(rvest)
library(jsonlite)
library(PortfolioAnalytics)
#library(quadprog)
library(ROI)
#library(ROI.plugin.glpk)
#library(ROI.plugin.quadprog)


**Task 2 - Markowitz portfolio (4 pts)**  
Using the data of the 20 stocks illustrate the portfolio performance by forming an efficient frontier. For the analysis, use either

20 random stocks from the data you have downloaded, i.e. stocks from "group_number_rand_download.csv" file
20 random stocks from the seminar, i.e. stocks from "group_number_rand_seminar.csv" file
choose this option in case you were not able to download data in Task 1
In this task, you will form 2 portfolios

portfolio A will consist of stocks 1 to 10
portfolio B will consist of stocks 11 to 20
Using the adjusted close price construct (and plot in the single figure) global minimum variance portfolio and the efficient frontier of both portfolio A and B. Since we will be working with monthly returns later in the course, use the monthly returns from the period 2015-2022 as the input for the task. Comment the important features of the figure, e.g. which portfolio will you choose? Why? Which stock(s) is driving the shape of the frontier? etc.

**(1). Reload data from Task 1**

In [ ]:
load("1-price.RData")

In [ ]:
symb <- attributes(OHLCV)$names
symb

**(2). Set Date Range**

In [ ]:
# Function of set date range for xts
setRange <- function(xts_in, date_in){    
    return (xts_in[index(xts_in) > date_in])
}

# Call the function and output updated links
OHLCV_2015 <- lapply(OHLCV, setRange, as.Date("2015-01-01"))
head(OHLCV_2015[[1]])
tail(OHLCV_2015[[1]])

**(3). filter for adjusted close price.**

In [ ]:
# only need the adjusted closing price for return

OHLCV_p <- lapply(names(OHLCV_2015), function(y){
    OHLCV_2015[[y]] <- OHLCV_2015[[y]][, paste0(y, '.Adjusted')]
})
names(OHLCV_p) <- names(OHLCV_2015)

# check columns
head(OHLCV_p[[1]])


**(4). Set monthly data**


In [ ]:
# Call to.period function
OHLCV_m <- lapply(OHLCV_p,to.period, period = 'months',name=NULL)

head(OHLCV_m[[1]])


**(5). calculate monthly return**
$$r_{i,t}=\frac{P_{i,t}-P_{i,t-1}}{P_{i,t-1}}$$

In [ ]:
# Selected Close Price
OHLCV_c <- lapply(names(OHLCV_m), function(y){
    OHLCV_m[[y]] <- OHLCV_m[[y]][, 'Close']
})
names(OHLCV_c) <- names(OHLCV_m)

# compute simple returns
rets <- lapply(OHLCV_c, function(y){
  y <- na.omit(diff(y)/lag(y))
})

head(rets[[1]])


**(6). Create Portflio data**

In [ ]:
# n_list is the whole 20 stocks portfolio
n_list <- do.call(merge, rets)
colnames(n_list) <- names(rets)

head(n_list)

**(7). Obtain Global Minimum Variance Portlio**

In [ ]:
# Create the portfolio 1 as the first 10 stocks.
returns <- n_list[, 1:10]
funds <- colnames(returns)

# Create an initial portfolio object with leverage and box constraints.
initial_portfolio <- portfolio.spec(assets = funds)
initial_portfolio <- add.constraint(
    portfolio = initial_portfolio,
    type = "leverage",
    min_sum = 0.99,
    max_sum = 1.01
)

# Obtain optimized portfolio by optimize.portfolio function
minvar <- add.objective(portfolio = initial_portfolio, type = "risk", name = "var")
opt_minvar <- optimize.portfolio(
    R = returns,
    portfolio = minvar,
    optimize_method = "ROI",
    trace = TRUE
)

print(opt_minvar)

**(8). Analysis portfolio 1**


In [ ]:
# Show each stock analsis in plot
plot(
    opt_minvar,
    risk.col = "StdDev",
    return.col = "mean",
    main = "Minimum Variance Optimization",
    chart.assets = TRUE,
    xlim = c(0.03, 0.12),
    ylim = c(0, 0.025)
)


**(9). Draw Efficient frontier**

In [ ]:
# Plot the efficient frontier.
meanvar <- create.EfficientFrontier(R = returns, portfolio = initial_portfolio, type = 'mean-var')
chart.EfficientFrontier(meanvar, match.col = 'StdDev', type = 'l', RAR.text = 'Sharpe Ratio', pch = 4)